In [106]:
from datetime import datetime, timedelta  # Import timedelta from datetime
import pandas as pd

Fin-Bert

In [107]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
model = pipeline("text-classification", model="ProsusAI/finbert")

Data preparation

In [108]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [109]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def scrape_news(company, pages = 1):

    # URL of the website to scrape
    columns = ['datetime', 'title', 'source', 'link']
    df = pd.DataFrame(columns=columns)

    for page in range(1, pages+1):
        url = f'https://markets.businessinsider.com/news/{company}-stock?p={page}'

        
        response = requests.get(url)
        html = response.text
        soup = BeautifulSoup(html, 'lxml')

        articles = soup.find_all('div', class_='latest-news__story')

        for article in articles:
            datetime = article.find('time', class_='latest-news__date').get('datetime')
            title = article.find('a', class_='news-link').text
            source = article.find('span', class_='latest-news__source').text
            link = article.find('a', class_='news-link').get('href')

            df = pd.concat([pd.DataFrame([[datetime, title, source, link]], columns=df.columns), df], ignore_index=True)

            
    return df


In [110]:
# a = soup.find_all('div', class_='latest-news__story')
# len(a)

In [118]:
df = scrape_news('aapl',300)

In [119]:
len(df)

15000

In [120]:
# save scraped_news
df.to_csv('scraped_news.csv', index=False)

In [121]:
# remove time from datetime
df['datetime'] = pd.to_datetime(df['datetime']).dt.date
df.set_index('datetime', inplace=True)

C:\Users\Ishit\AppData\Local\Temp\ipykernel_18084\3893246076.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['datetime']).dt.date


In [150]:
# df['sentiment'] = df['title'].apply(lambda x: model(x)[0]['label'])
# df['score'] = df['title'].apply(lambda x: model(x)[0]['score'])

# df["positive"] = 0
# df["negative"] = 0
# df["neutral"] = 0

In [122]:
# find missing dates
missing_dates = pd.date_range(start=df.index.min(), end=df.index.max()).difference(df.index)

In [123]:
missing_dates

DatetimeIndex(['2018-05-20', '2018-05-28', '2018-06-03', '2018-06-07',
               '2018-06-10', '2018-06-24', '2018-07-01', '2018-07-04',
               '2018-07-06', '2018-07-07',
               ...
               '2024-08-30', '2024-08-31', '2024-09-01', '2024-09-02',
               '2024-09-07', '2024-09-08', '2024-09-14', '2024-09-15',
               '2024-09-28', '2024-09-29'],
              dtype='datetime64[ns]', length=302, freq=None)

In [124]:
df.source.unique()

array(['Business Insider', 'Reuters', 'RTTNews', 'PR Newswire',
       'Markets Insider Automation', 'Premium', 'Moodys', 'InvestorPlace',
       'Seeking Alpha', 'Benzinga', 'TipRanks', 'GlobeNewswire'],
      dtype=object)

In [125]:
df.index.min(), df.index.max()

(datetime.date(2018, 5, 17), datetime.date(2024, 10, 4))

In [126]:
df.head()

,title,source,link
datetime,,,
2018-05-17,The rapid-fire way Steve Jobs got employees to...,Business Insider,http://www.businessinsider.com/steve-jobs-pixa...
2018-05-18,Apple pays Ireland first tranche of disputed t...,Reuters,http://feeds.reuters.com/~r/reuters/companyNew...
2018-05-18,CORRECTED-Apple pays Ireland first tranche of ...,Reuters,http://feeds.reuters.com/~r/reuters/companyNew...
2018-05-19,Apple fans are raging that the new MacBook Pro...,Business Insider,http://www.businessinsider.com/apple-macbook-p...
2018-05-21,Apple just cut the price of the one cable you ...,Business Insider,http://www.businessinsider.com/apple-usb-c-to-...


In [157]:
# mode_sentiment = df.groupby(df.index)['sentiment'].agg(lambda x: x.mode().tolist())

In [12]:
# mode_sentiment.value_counts()

In [127]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [128]:
import torch
from torch import nn

def classify_sentiment(text_list, batch_size=128):
    all_predictions = []
    all_pos_scores = []
    all_neg_scores = []
    all_neu_scores = []

    # Process in batches
    for i in range(0, len(text_list), batch_size):
        pt_batch = tokenizer(
            text_list[i:i + batch_size],  # Get a batch of texts
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        with torch.no_grad():  # Disable gradient calculation
            pt_outputs = model(**pt_batch)
            pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=1)

        max_prediction = torch.argmax(pt_predictions, dim=1).tolist()
        max_scores = torch.max(pt_predictions, dim=1).values.tolist()

        # Adjust predictions and scores
        # for j in range(len(max_prediction)):
        #     if max_prediction[j] == 2:  # Check for neutral sentiment
        #         max_prediction[j] = 0 if pt_predictions[j][0] > pt_predictions[j][1] else 1
        #         max_scores[j] = pt_predictions[j][0] if pt_predictions[j][0] > pt_predictions[j][1] else pt_predictions[j][1]

        # Extend results
        positive_score = [pt_predictions[j][0].item() for j in range(len(max_prediction))]
        negative_score = [pt_predictions[j][1].item() for j in range(len(max_prediction))]
        neutral_score = [pt_predictions[j][2].item() for j in range(len(max_prediction))]


        all_predictions.extend(max_prediction)
        # all_scores.extend(max_scores)
        all_pos_scores.extend(positive_score)
        all_neg_scores.extend(negative_score)
        all_neu_scores.extend(neutral_score)



    return all_predictions, all_pos_scores, all_neg_scores, all_neu_scores




In [129]:
x,y,z,t = classify_sentiment(["high earnings","low"])

In [130]:
# detached_tensors = [t.detach().item() for t in y]
# detached_tensors
x,y,z,t

([2, 2],
 [0.3832497000694275, 0.129344642162323],
 [0.039265163242816925, 0.05032506585121155],
 [0.577485203742981, 0.8203302621841431])

In [131]:
sentiment, pos_score, neg_score, neu_score = classify_sentiment(df['title'].tolist())

In [24]:
# pos_score

In [132]:
df['sentiment'] = sentiment
df['positive'] = pos_score
df['negative'] = neg_score
df['neutral'] = neu_score

# df['scores']  = [t.detach().item() if isinstance(t, torch.Tensor) else t for t in scores]

In [133]:
# df['sentiment'] = x
# scores

In [134]:
# df['scores'][-150:]

In [135]:
mapping = {0:'positive',1:'negative',2:'neutral'}
df['sentiment'] = df['sentiment'].map(mapping)

In [136]:
df.head()

,title,source,link,sentiment,positive,negative,neutral
datetime,,,,,,,
2018-05-17,The rapid-fire way Steve Jobs got employees to...,Business Insider,http://www.businessinsider.com/steve-jobs-pixa...,neutral,0.032691,0.074973,0.892336
2018-05-18,Apple pays Ireland first tranche of disputed t...,Reuters,http://feeds.reuters.com/~r/reuters/companyNew...,neutral,0.056335,0.025111,0.918554
2018-05-18,CORRECTED-Apple pays Ireland first tranche of ...,Reuters,http://feeds.reuters.com/~r/reuters/companyNew...,neutral,0.073576,0.056256,0.870168
2018-05-19,Apple fans are raging that the new MacBook Pro...,Business Insider,http://www.businessinsider.com/apple-macbook-p...,negative,0.022010,0.703944,0.274046
2018-05-21,Apple just cut the price of the one cable you ...,Business Insider,http://www.businessinsider.com/apple-usb-c-to-...,neutral,0.140317,0.324263,0.535420


In [137]:
# filtered_df = df[df.index == pd.to_datetime('2024-09-9').date() ]
# filtered_df.head()
# df['title'][1]

In [138]:
# len(filtered_df["sentiment"].mode())

In [139]:
# filtered_df["sentiment"].mode()[0]
# pos_score = filtered_df[ filtered_df['sentiment'] == 'positive']["scores"].sum()
# neg_score = filtered_df[ filtered_df['sentiment'] == 'negative']["scores"].sum()

In [140]:
# neg_score = filtered_df[ filtered_df['sentiment'] == 'negative']["scores"].sum()

In [141]:
pd.to_datetime('2024-07-05').date() == df.index.min()

False

In [142]:
def get_sentiment_score(start_date, end_date):
    
    filtered_df = df[(df.index >= start_date) & (df.index <= end_date)]
    
    if filtered_df.empty:
        return 'neutral', 0, 0, 0

    pos_score = filtered_df["positive"].mean()
    neg_score = filtered_df["negative"].mean()
    neu_score = filtered_df["neutral"].mean()

    mode_sentiment = filtered_df["sentiment"].mode()
    print(mode_sentiment)
    
    if len(mode_sentiment) == 0:
        return 'neutral', 0, 0, 0
    
    if len(mode_sentiment) > 1:
        # create a set of sentiment
        sentiment_set = set(mode_sentiment)

        if 'positive' in sentiment_set:
            if 'negative' in sentiment_set:
                return ('positive', pos_score, neg_score, neu_score) if pos_score > neg_score else ('negative', pos_score, neg_score, neu_score)
            elif 'neutral' in sentiment_set:
                return ('positive', pos_score, neg_score, neu_score) if pos_score > neu_score else ('neutral', pos_score, neg_score, neu_score)
            
        return ('negative', pos_score, neg_score, neu_score) if neg_score > neu_score else ('neutral', pos_score, neg_score, neu_score)

    return mode_sentiment[0], pos_score, neg_score, neu_score


In [99]:
start_date = pd.to_datetime('2024-07-25').date()
get_sentiment_score(start_date, start_date)

0     neutral
1    positive
Name: sentiment, dtype: object


('positive', 0.5196308543284734, 0.038305554228524365, 0.4420635898907979)

In [150]:
import yfinance as yf

ticker = 'AAPL'
start_date = '2018-05-20'
end_date = '2024-10-05'

stock_data = yf.download(ticker, start=start_date, end=end_date)
stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d').date

[*********************100%***********************]  1 of 1 completed


In [151]:
stock_data['sentiment'] = None
stock_data['pos_score'] = 0
stock_data['neg_score'] = 0
stock_data['neu_score'] = 0
stock_data.head()

,Open,High,Low,Close,Adj Close,Volume,sentiment,pos_score,neg_score,neu_score
2018-05-21,47.000000,47.317501,46.727501,46.907501,44.539970,73603200,None,0,0,0
2018-05-22,47.095001,47.220001,46.695000,46.790001,44.428417,60962800,None,0,0,0
2018-05-23,46.587502,47.125000,46.439999,47.090000,44.713261,80233600,None,0,0,0
2018-05-24,47.192501,47.209999,46.552502,47.037498,44.663403,92936000,None,0,0,0
2018-05-25,47.057499,47.412498,46.912498,47.145000,44.765484,69844000,None,0,0,0


In [152]:
for i in range(len(stock_data)):
    if i == 0:
        start_date = stock_data.index[i] - timedelta(days=2)
        end_date = stock_data.index[i] - timedelta(days=1)
        
        sentiment, pos, neg, neu = get_sentiment_score(start_date, end_date)
        
        # Use iloc for positional indexing
        stock_data.iloc[i, stock_data.columns.get_loc('sentiment')] = sentiment
        stock_data.iloc[i, stock_data.columns.get_loc('pos_score')] = pos
        stock_data.iloc[i, stock_data.columns.get_loc('neg_score')] = neg
        stock_data.iloc[i, stock_data.columns.get_loc('neu_score')] = neu
        
        # print(stock_data.iloc[i, stock_data.columns.get_loc('sentiment')], stock_data.iloc[i, stock_data.columns.get_loc('score')])
    else:
        start_date = stock_data.index[i-1]
        end_date = stock_data.index[i] - timedelta(days=1)
        print(start_date, end_date)
        print(get_sentiment_score(start_date, end_date))
        sentiment, pos, neg, neu = get_sentiment_score(start_date, end_date)
        
        # Use iloc for positional indexing
        stock_data.iloc[i, stock_data.columns.get_loc('sentiment')] = sentiment
        stock_data.iloc[i, stock_data.columns.get_loc('pos_score')] = pos
        stock_data.iloc[i, stock_data.columns.get_loc('neg_score')] = neg
        stock_data.iloc[i, stock_data.columns.get_loc('neu_score')] = neu
        
        # print(stock_data.iloc[i, stock_data.columns.get_loc('sentiment')], stock_data.iloc[i, stock_data.columns.get_loc('score')])
    print(i*"-")

0    negative
Name: sentiment, dtype: object

2018-05-21 2018-05-21
0    neutral
Name: sentiment, dtype: object
('neutral', 0.10073424875736237, 0.1720678685232997, 0.727197915315628)
0    neutral
Name: sentiment, dtype: object
-
2018-05-22 2018-05-22
0    neutral
Name: sentiment, dtype: object
('neutral', 0.03185869753360748, 0.24994856119155884, 0.7181926965713501)
0    neutral
Name: sentiment, dtype: object
--
2018-05-23 2018-05-23
0    neutral
Name: sentiment, dtype: object
('neutral', 0.07028272189199924, 0.018399101682007313, 0.9113181829452515)
0    neutral
Name: sentiment, dtype: object
---
2018-05-24 2018-05-24
0    neutral
Name: sentiment, dtype: object
('neutral', 0.28492236603051424, 0.12087754098077615, 0.5942000970244408)
0    neutral
Name: sentiment, dtype: object
----
2018-05-25 2018-05-28
0    positive
Name: sentiment, dtype: object
('positive', 0.40178920413282787, 0.2815228450874036, 0.31668793477795343)
0    positive
Name: sentiment, dtype: object
-----
2018-05-29 2

C:\Users\Ishit\AppData\Local\Temp\ipykernel_18084\1711551278.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.022010313346982002' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  stock_data.iloc[i, stock_data.columns.get_loc('pos_score')] = pos
C:\Users\Ishit\AppData\Local\Temp\ipykernel_18084\1711551278.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.7039440870285034' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  stock_data.iloc[i, stock_data.columns.get_loc('neg_score')] = neg
C:\Users\Ishit\AppData\Local\Temp\ipykernel_18084\1711551278.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.27404558658599854' has dtype incompatible with int64, p

0    negative
Name: sentiment, dtype: object
('negative', 0.11442182441665368, 0.5091740407726981, 0.3764041033996777)
0    negative
Name: sentiment, dtype: object
-------------------------------------------------------------------------------------------------------
2018-10-16 2018-10-16
0    negative
Name: sentiment, dtype: object
('negative', 0.13021256949286908, 0.4714724461082369, 0.3983149800915271)
0    negative
Name: sentiment, dtype: object
--------------------------------------------------------------------------------------------------------
2018-10-17 2018-10-17
0    neutral
Name: sentiment, dtype: object
('neutral', 0.23571902296195427, 0.16114158276468515, 0.6031393756469091)
0    neutral
Name: sentiment, dtype: object
---------------------------------------------------------------------------------------------------------
2018-10-18 2018-10-18
0    neutral
Name: sentiment, dtype: object
('neutral', 0.05525900237262249, 0.019429598934948444, 0.925311416387558)
0    neutra

In [103]:
# start_date = pd.to_datetime('2024-07-25').date()
# get_sentiment_score(start_date, end_date)

In [146]:
stock_data[:10]

,Open,High,Low,Close,Adj Close,Volume,sentiment,pos_score,neg_score,neu_score
2021-05-20,125.230003,127.720001,125.099998,127.309998,125.025925,76857100,neutral,0.175629,0.281067,0.543304
2021-05-21,127.820000,128.000000,125.209999,125.430000,123.179649,79295400,positive,0.507509,0.224136,0.268355
2021-05-24,126.010002,127.940002,125.940002,127.099998,124.819687,63092900,neutral,0.206313,0.205134,0.588553
2021-05-25,127.820000,128.320007,126.320000,126.900002,124.623268,72009500,neutral,0.343296,0.094864,0.561841
2021-05-26,126.959999,127.389999,126.419998,126.849998,124.574165,56575900,neutral,0.063131,0.366664,0.570205
2021-05-27,126.440002,127.639999,125.080002,125.279999,123.032341,94625600,neutral,0.085336,0.168749,0.745915
2021-05-28,125.570000,125.800003,124.550003,124.610001,122.374367,71311100,neutral,0.119771,0.028907,0.851322
2021-06-01,125.080002,125.349998,123.940002,124.279999,122.050278,67637100,neutral,0.142534,0.299404,0.558062
2021-06-02,124.279999,125.239998,124.050003,125.059998,122.816269,59278900,neutral,0.096011,0.256904,0.647084
2021-06-03,124.680000,124.849998,123.129997,123.540001,121.323555,76229200,neutral,0.148307,0.016348,0.835345


In [153]:
stock_data.tail()

,Open,High,Low,Close,Adj Close,Volume,sentiment,pos_score,neg_score,neu_score
2024-09-30,230.039993,233.000000,229.649994,233.000000,233.000000,54541900,neutral,0.476180,0.012793,0.511027
2024-10-01,229.520004,229.649994,223.740005,226.210007,226.210007,63285000,neutral,0.360028,0.258625,0.381347
2024-10-02,225.889999,227.369995,223.020004,226.779999,226.779999,32880600,negative,0.150119,0.444031,0.405850
2024-10-03,225.139999,226.809998,223.320007,225.669998,225.669998,34044200,positive,0.945762,0.016205,0.038033
2024-10-04,227.899994,228.000000,224.130005,226.800003,226.800003,37245100,positive,0.743496,0.009269,0.247235


In [154]:
len(stock_data)

1605

In [155]:
stock_data.sentiment.value_counts()

sentiment
neutral     1313
negative     167
positive     125
Name: count, dtype: int64

In [156]:
stock_data.to_csv('stock_data_final.csv')